In [1]:
from vector_database import *

c:\Users\Pc\AppData\Local\pypoetry\Cache\virtualenvs\eve-project-13oD41wO-py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pdfs_folder_path = 'pdfs'
db_path = './pdftst'
vector_creator = VectorCreator(pdfs_folder_path=pdfs_folder_path, db_path=db_path)
vector_creator.create_from_pdfs()

100%|██████████| 1/1 [00:09<00:00,  9.56s/it]


In [9]:
pdf_path = r'pdfs\Anonymous_health_and_wellbeing_Survey.pdf'

import pypdf
pdf = pypdf.PdfReader(open(pdf_path, 'rb'))

In [15]:
print(pdf.pages[0].extract_text())

Anonymous health and wellbeing  
Survey Y2 2023  
The survey will take approximately 4 minutes to complete.
This is anonymous , your answers is not linked to your email address.  This to ensure  
confidentiality.
Officer / Chief / SL
Operator / Rating
OtherWhat is your position?1.


In [183]:
pages = [pdf.pages[i].extract_text() for i in range(len(pdf.pages))]
pages[0] = pages[0][213:]
# delete pages[-1]
pages.pop(-1)

'This content is neither created nor endorsed by Microsoft. The data you submit will be sent to the form owner.\nMicrosoft FormsNot true at all\nRarely true\nSometimes true\nOften true\nTrue nearly all the timeI am able to adapt when changes occur30.\nNot true at all\nRarely true\nSometimes true\nOften true\nTrue nearly all the timeI tend to bounce back after illness, injury or other hardships31.'

In [243]:
import re

def separate_questions_answers(text):
    pattern = r'([A-Z].*?)\d+\.'  # Patrón actualizado para coincidir con preguntas
    matches = re.findall(pattern, text)

    ixq = [[i for i, letter in enumerate(matches[j]) if letter.isupper()][-1] for j, _ in enumerate(matches)]
    questions = [question[ixq[j]:] for j, question in enumerate(matches)]    
    
    answers = []
    splited = text.split('\n')
    # lines that contain a question
    idx_questions = [i for i, line in enumerate(splited) if any(question in line for question in questions)]

    for i, question in enumerate(questions):
        if i == 0:
            ans = splited[:idx_questions[i]]
            ans.append(splited[idx_questions[i]].split(question)[0])
        else:
            ans = splited[idx_questions[i-1]+1:idx_questions[i]]
            ans.append(splited[idx_questions[i]].split(question)[0])

        answers.append(ans)

    return questions, answers


In [244]:
questions = []
answers = []

for page in pages:
    fixed_page = [line if len(line)<74 else (line if (i==len(line) or i+1>=len(page.split('\n'))) else line + page.split('\n')[i+1]) for i, line in enumerate(page.split('\n'))]
    itst = [i+1 for i, line in enumerate(fixed_page) if len(line)>70]
    # elimino los elementos con indices itst
    fixed_page = [line for i, line in enumerate(fixed_page) if i not in itst]
    fixed_page = [re.sub(r'\s{2,}', '', line) for line in fixed_page]
    fixed_page = '\n'.join(fixed_page)
    q, a = separate_questions_answers(fixed_page)
    questions.extend(q)
    answers.extend(a)

In [237]:
questions

['What is your position?',
 'What is your age?',
 'What shift are you on?',
 'How many years of seismic offshore experience do you have?',
 'Which vessel do you work on?',
 'How many hours per day (on - and off shift) do you spend sitting orphysically inactive?',
 'How often do you go outside to get some fresh air during a day?',
 'How often do you do exercise in your time off per week?',
 'How many servings of fruit and vegetables do you usually eat per day (athome and at work)?',
 'How many days a week do you usually eat fast food or food that is highin fat and salt (at home and at work)?',
 'When was the last time you had your cholesterol measured?',
 'When was the last time you had your blood pressuremeasured?',
 'I feel unwell.',
 'In the last month, how often have you felt confident about your ability tohandle your personal problems?',
 'In the last month, how often have you felt that things were going yourway?',
 'In the last month, how often have you felt difficulties were pili

In [245]:
answers

[['Officer / Chief / SL', 'Operator / Rating', 'Other'],
 ['Under 24', '25 - 34', '35 - 44', '45 - 54', 'Over 55'],
 ['00 - 12', '12 - 24', 'Dayshift', 'Other'],
 ['1 year or less',
  '2 - 4 years',
  '5 - 9 years',
  '10 - 15 years',
  'Over 15 years'],
 ['Tethys',
  'Sovereign',
  'Vanguard',
  'Atlas',
  'Hyperion',
  'Tansa',
  'Apollo',
  'Swift',
  'Victory'],
 ['Less than 2 hrs', '3 - 4 hrs', '5 - 7 hrs', '8 - 11 hrs', 'Over 11 hrs'],
 ['Never', 'Almost never', 'Sometimes', 'Fairly often', 'Very often'],
 ['Never',
  '0 - 60 minutes',
  '61 - 100 minutes',
  '101 - 149 minutes',
  '150 - 300 minutes',
  'More than 300 minutes'],
 ['1 or less', '2 servings', '3 servings', '4 servings', '5 or more'],
 ['None', '1', '2 - 3 times', '4 - 5 times', '5 or more'],
 ['Never', 'Last month', '6 months', '1 year', 'Over 2 years'],
 ['Last month', '6 months', '1 year', 'Over 2 years'],
 ['Strongly agree',
  'Agree',
  'Neutral',
  'Disagree',
  'Strongly disagreeI’m comfortable seeing the Me

In [238]:
len(questions)

25

In [145]:
len(pages[4].split('\n')[5])

76

In [146]:
len(pages[4].split('\n')[11])

83

In [225]:
fixed_page = [line if len(line)<75 else (line if (i==len(line) or i+1>=len(pages[4].split('\n'))) else line + pages[4].split('\n')[i+1]) for i, line in enumerate(pages[4].split('\n'))]
itst = [i+1 for i, line in enumerate(fixed_page) if len(line)>70]
# elimino los elementos con indices itst
fixed_page = [line for i, line in enumerate(fixed_page) if i not in itst]
fixed_page = [re.sub(r'\s{2,}', '', line) for line in fixed_page]

In [226]:
fixed_page

['Never',
 '0 - 60 minutes',
 '61 - 100 minutes',
 '101 - 149 minutes',
 '150 - 300 minutes',
 'More than 300 minutesHow often do you do exercise in your time off per week?8.',
 '1 or less',
 '2 servings',
 '3 servings',
 '4 servings',
 '5 or moreHow many servings of fruit and vegetables do you usually eat per day (athome and at work)?9.']

In [150]:
itst

[6, 12]

In [148]:
fixed_page

['Never',
 '0 - 60 minutes',
 '61 - 100 minutes',
 '101 - 149 minutes',
 '150 - 300 minutes',
 'More than 300 minutesHow often do you do exercise in your time off per week?      8.',
 '      8.',
 '1 or less',
 '2 servings',
 '3 servings',
 '4 servings',
 '5 or moreHow many servings of fruit and vegetables do you usually eat per day (at  home and at work)?9.',
 'home and at work)?9.']

In [160]:
# uno tst con \n
fixed_page = '\n'.join(fixed_page)

In [161]:
fixed_page

'Never\n0 - 60 minutes\n61 - 100 minutes\n101 - 149 minutes\n150 - 300 minutes\nMore than 300 minutesHow often do you do exercise in your time off per week?8.\n1 or less\n2 servings\n3 servings\n4 servings\n5 or moreHow many servings of fruit and vegetables do you usually eat per day (athome and at work)?9.'

In [227]:
separate_questions_answers(pages[-1])

(['I usually get the following hours of sleep'],
 [['Never',
   'Almost never',
   'Sometimes',
   'Fairly often',
   'Very oftenHow often are you distracted or find it difficult to concentrate?',
   '      28.',
   '0-2',
   '3-4',
   '5-6',
   '7-8',
   '9-10On an average day, ']])

In [228]:
print(pages[-1])

Never
Almost never
Sometimes
Fairly often
Very oftenHow often are you distracted or find it difficult to concentrate?
      28.
0-2
3-4
5-6
7-8
9-10On an average day, I usually get the following hours of sleep29.


In [229]:
len(pages[-1].split('\n')[4])

75

In [190]:
fixed_page = [line if len(line)<76 else line + pages[-1].split('\n')[i+1] for i, line in enumerate(pages[-1].split('\n'))]
itst = [i+1 for i, line in enumerate(fixed_page) if len(line)>70]
# elimino los elementos con indices itst
fixed_page = [line for i, line in enumerate(fixed_page) if i not in itst]
fixed_page = [re.sub(r'\s{2,}', '', line) for line in fixed_page]
fixed_page = '\n'.join(fixed_page)

In [204]:
fixed_page = [line if len(line)<75 else line + pages[-1].split('\n')[i+1] for i, line in enumerate(pages[-1].split('\n'))]


In [203]:
len(pages[-1].split('\n')[4])

75

In [205]:
print(fixed_page)

['Never', 'Almost never', 'Sometimes', 'Fairly often', 'Very oftenHow often are you distracted or find it difficult to concentrate?      28.', '      28.', '0-2', '3-4', '5-6', '7-8', '9-10On an average day, I usually get the following hours of sleep29.']


In [ ]:
tt = {}
tt

In [246]:
{'question': 'What is your position?', 'answers': ['Officer / Chief / SL', 'Operator / Rating', 'Other']},
 {'question': 'What is your age?', 'answers': ['Under 24', '25 - 34', '35 - 44', '45 - 54', 'Over 55']}

TypeError: unhashable type: 'dict'